# Business Probelm
If you choose the Recommendation System option, you will be making movie recommendations based on the MovieLensLinks to an external site. dataset from the GroupLens research lab at the University of Minnesota. Unless you are planning to run your analysis on a paid cloud platform, we recommend that you use the "small" dataset containing 100,000 user ratings (and potentially, only a particular subset of that dataset).

Your task is to:

Build a model that provides top 5 movie recommendations to a user, based on their ratings of other movies.
The MovieLens dataset is a "classic" recommendation system dataset, that is used in numerous academic papers and machine learning proofs-of-concept. You will need to create the specific details about how the user will provide their ratings of other movies, in addition to formulating a more specific business problem within the general context of "recommending movies".

Collaborative Filtering
At minimum, your recommendation system must use collaborative filtering. If you have time, consider implementing a hybrid approach, e.g. using collaborative filtering as the primary mechanism, but using content-based filtering to address the cold start problemLinks to an external site..

Evaluation
The MovieLens dataset has explicit ratings, so achieving some sort of evaluation of your model is simple enough. But you should give some thought to the question of metrics. Since the rankings are ordinal, we know we can treat this like a regression problem. But when it comes to regression metrics there are several choices: RMSE, MAE, etc. HereLinks to an external site. are some further ideas.

In [2]:

import numpy as np
import pandas as pd

In [3]:
#Importing Libraries
df = pd.concat(
    map(pd.read_csv, ['data/movies.csv', 'data/links.csv', 'data/ratings.csv', 'data/tags.csv']), ignore_index=True)
print(df)

        movieId                               title  \
0             1                    Toy Story (1995)   
1             2                      Jumanji (1995)   
2             3             Grumpier Old Men (1995)   
3             4            Waiting to Exhale (1995)   
4             5  Father of the Bride Part II (1995)   
...         ...                                 ...   
123998     7382                                 NaN   
123999     7936                                 NaN   
124000     3265                                 NaN   
124001     3265                                 NaN   
124002   168248                                 NaN   

                                             genres  imdbId  tmdbId  userId  \
0       Adventure|Animation|Children|Comedy|Fantasy     NaN     NaN     NaN   
1                        Adventure|Children|Fantasy     NaN     NaN     NaN   
2                                    Comedy|Romance     NaN     NaN     NaN   
3                      

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124003 entries, 0 to 124002
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    124003 non-null  int64  
 1   title      9742 non-null    object 
 2   genres     9742 non-null    object 
 3   imdbId     9742 non-null    float64
 4   tmdbId     9734 non-null    float64
 5   userId     104519 non-null  float64
 6   rating     100836 non-null  float64
 7   timestamp  104519 non-null  float64
 8   tag        3683 non-null    object 
dtypes: float64(5), int64(1), object(3)
memory usage: 8.5+ MB


In [5]:
df['rating'].value_counts()
# ratings range from .5 - 4.

4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: int64

In [6]:
df['genres'].value_counts()

Drama                                          1053
Comedy                                          946
Comedy|Drama                                    435
Comedy|Romance                                  363
Drama|Romance                                   349
                                               ... 
Drama|Mystery|Romance|Sci-Fi|Thriller             1
Action|Adventure|Drama|Fantasy|Mystery|IMAX       1
Animation|Fantasy|Sci-Fi|War                      1
Adventure|Fantasy|Thriller|IMAX                   1
Action|Comedy|Crime|Drama|Horror|Thriller         1
Name: genres, Length: 951, dtype: int64

In [7]:
df['title'].unique()

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'Bungo Stray Dogs: Dead Apple (2018)',
       'Andrew Dice Clay: Dice Rules (1991)', nan], dtype=object)

In [8]:
df.title.value_counts()

Saturn 3 (1980)                           2
Confessions of a Dangerous Mind (2002)    2
Emma (1996)                               2
War of the Worlds (2005)                  2
Eros (2004)                               2
                                         ..
The Emperor's Club (2002)                 1
Reform School Girls (1986)                1
Dr. Phibes Rises Again (1972)             1
Serendipity (2001)                        1
Little Rascals, The (1994)                1
Name: title, Length: 9737, dtype: int64

In [9]:
df.head()

,movieId,title,genres,imdbId,tmdbId,userId,rating,timestamp,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Jumanji (1995),Adventure|Children|Fantasy,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Grumpier Old Men (1995),Comedy|Romance,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Father of the Bride Part II (1995),Comedy,NaN,NaN,NaN,NaN,NaN,NaN
